In [ ]:
!pip install pandas
!pip install torch
!pip install transformers
!pip install scikit-learn
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post5-py3-none-any.whl size=2950 sha256=7025917e314030729e9576d0317b5f9b652b77d4226950aeaacef0e

In [ ]:
!pip install accelerate -U
!pip install transformers[torch]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 11.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Change the current working directory to the desired directory in Google Drive
os.chdir('/content/drive/MyDrive/CS224U')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast, RobertaTokenizerFast
import torch


In [ ]:

# Load the data
df = pd.read_csv('semeval.csv')
df.rename(columns = {'tweet': 'text'}, inplace=True)

sentiment_mapping = {
    'positive': 1,
    'negative': 0,
    'neutral': 2
}
df['sentiment'] = df['sentiment'].map(sentiment_mapping)

# Initial split: 60% for training, 40% for a temporary set
df_train, df_temp = train_test_split(df, train_size=0.6)

# Split the temporary set evenly into the testing and development sets
df_dev, df_test = train_test_split(df_temp, train_size=0.5)


### ADDED CODE
df_train.to_csv('train.csv', sep=',')
df_dev.to_csv('dev.csv', sep=',')

df_test = df_test.sample(1000)
df_test.to_csv('test.csv', sep=',')


### End

In [ ]:
### added code
df_train = pd.read_csv('train.csv')
df_dev = pd.read_csv('dev.csv')
df_test = pd.read_csv('final-translated_sentiment.csv')

# end


# Split into training and testing sets
train_texts, dev_texts, test_texts, train_labels, dev_labels, test_labels = df_train['text'], df_dev['text'], df_test['predictions'], df_train['sentiment'], df_dev['sentiment'], df_test['sentiment']

# Initialize tokenizers
bert_tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
roberta_tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Tokenize the texts
train_encodings_bert = bert_tokenizer(list(train_texts), truncation=True, padding=True)
dev_encodings_bert = bert_tokenizer(list(dev_texts), truncation=True, padding=True)
test_encodings_bert = bert_tokenizer(list(test_texts), truncation=True, padding=True)

train_encodings_roberta = roberta_tokenizer(list(train_texts), truncation=True, padding=True)
dev_encodings_roberta = roberta_tokenizer(list(dev_texts), truncation=True, padding=True)
test_encodings_roberta = roberta_tokenizer(list(test_texts), truncation=True, padding=True)

# Create a dataloader
class TwitterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create training and testing datasets
train_dataset_bert = TwitterDataset(train_encodings_bert, train_labels)
dev_dataset_bert = TwitterDataset(dev_encodings_bert, dev_labels)
test_dataset_bert = TwitterDataset(test_encodings_bert, test_labels)

train_dataset_roberta = TwitterDataset(train_encodings_roberta, train_labels)
dev_dataset_roberta = TwitterDataset(dev_encodings_roberta, dev_labels)
test_dataset_roberta = TwitterDataset(test_encodings_roberta, test_labels)

In [ ]:
num_labels = len(set(train_labels))
num_labels

3

In [ ]:
from transformers import Trainer, TrainingArguments
def train_model(model, train_dataset, val_dataset, model_path):

    training_args = TrainingArguments(
        output_dir='./results', 
        num_train_epochs=3,              
        per_device_train_batch_size=16,  
        per_device_eval_batch_size=64,   
        warmup_steps=500,                
        weight_decay=0.01,               
        logging_dir='./logs',
    )

    trainer = Trainer(
        model=model,                       
        args=training_args,                  
        train_dataset=train_dataset,         
        eval_dataset=val_dataset             
    )

    trainer.train()

    # Save the model
    model.save_pretrained(model_path)

In [ ]:
from transformers import BertForSequenceClassification, RobertaForSequenceClassification
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Initialize models
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should pr

In [ ]:

# Call the train_model function for each
model_path = "./model"
train_model(bert_model, train_dataset_bert, dev_dataset_bert, f'{model_path}/bert_model')
train_model(roberta_model, train_dataset_roberta, dev_dataset_roberta, f'{model_path}/roberta_model')

In [ ]:
from transformers import Trainer
#from setup import test_dataset_bert, test_dataset_roberta
#from train import bert_model, roberta_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)

    em = np.mean([np.array_equal(p, l) for p, l in zip(preds, labels)])
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'exact_match': em
    }

def evaluate_model(model, dataset):
    trainer = Trainer(model, compute_metrics=compute_metrics)
    result = trainer.evaluate(dataset)
    return result

In [ ]:
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)

bert_model.load_state_dict(torch.load('/content/drive/MyDrive/CS224U/bert_model.pth'))
roberta_model.load_state_dict(torch.load('/content/drive/MyDrive/CS224U/roberta_model.pth'))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

<All keys matched successfully>

In [ ]:
from transformers import BertForSequenceClassification, RobertaForSequenceClassification

# Initialize models
# bert_model = BertForSequenceClassification.from_pretrained('models/bert_model')
# roberta_model = RobertaForSequenceClassification.from_pretrained('models/roberta_model')

# Call the evaluate_model function
bert_result = evaluate_model(bert_model, test_dataset_bert)
roberta_result = evaluate_model(roberta_model, test_dataset_roberta)

print("BERT Evaluation Result: ", bert_result)
print("RoBERTa Evaluation Result: ", roberta_result)

BERT Evaluation Result:  {'eval_loss': 0.5982958078384399, 'eval_accuracy': 0.804, 'eval_f1': 0.8048929834544688, 'eval_precision': 0.8088692549335481, 'eval_recall': 0.804, 'eval_exact_match': 0.804, 'eval_runtime': 4.7018, 'eval_samples_per_second': 106.342, 'eval_steps_per_second': 13.399}
RoBERTa Evaluation Result:  {'eval_loss': 0.5384219884872437, 'eval_accuracy': 0.784, 'eval_f1': 0.7849919602576992, 'eval_precision': 0.7879290422064964, 'eval_recall': 0.784, 'eval_exact_match': 0.784, 'eval_runtime': 1.7664, 'eval_samples_per_second': 283.062, 'eval_steps_per_second': 35.666}


In [ ]:
torch.save(bert_model.state_dict(), "/content/drive/MyDrive/CS224U/bert_model.pth")
torch.save(roberta_model.state_dict(), "/content/drive/MyDrive/CS224U/roberta_model.pth")